In [29]:
# import libraries
import pandas as pd
import numpy as np
import datetime

In [30]:
# Load data
df = pd.ExcelFile("PD week 12 input.xlsx")

In [31]:
# there are 3 sheets to look
total_sales = pd.read_excel(df, "Total Sales")
percentage_sales = pd.read_excel(df, "Percentage of Sales")
lookup = pd.read_excel(df, "Lookup Table")

In [32]:
# final output requires the Date to be in the year week number format
percentage_sales["week_of_year"] = percentage_sales["Week Commencing"].apply(lambda xx: datetime.datetime.strptime(str(xx)[:10], "%Y-%m-%d").isocalendar()[1])
percentage_sales["year"] = percentage_sales["Week Commencing"].apply(lambda xx: str(xx)[:4])
percentage_sales["Year Week Number"] = percentage_sales[["year", "week_of_year"]].apply(lambda xx: int(str(xx[0]) + '{0}'.format(str(xx[1]).zfill(2))), axis=1)

In [33]:
# remove product sizes that make up 0% of sales
percentage_sales = percentage_sales[percentage_sales["Percentage of Sales"] > 0.0]

In [34]:
# In the Lookup Table, it seems the Product ID and Size have been erroneously concatenated. These need to be separated.

# but why come up with potentially incorrect logic to separate them? let's just concatenate product ID and size from another tab :)
percentage_sales["id"] = percentage_sales["Product ID"] + percentage_sales["Size"]

In [35]:
# merge!
merge_1 = pd.merge(percentage_sales, lookup, how="left", left_on="id", right_on="Product")

In [50]:
# You'll need to do some cleaning of the Scent fields to join together the Total Sales and the Lookup Table.
total_sales["Scent"] = total_sales["Scent"].apply(lambda x: "".join(x.split()).capitalize())
merge_1["Scent"] = merge_1["Scent"].apply(lambda x: "".join(x.split()))

In [53]:
# merge!
merge_2 = pd.merge(merge_1, total_sales, how="left", left_on=["Scent", "Year Week Number"], right_on=["Scent", "Year Week Number"])

# compute the weekly sale
merge_2["Sales"] = merge_2[["Total Scent Sales", "Percentage of Sales"]].apply(lambda x: round(x[0] * x[1], 2), axis=1)

In [54]:
merge_2.head(25)

,Product ID,Week Commencing,Size,Product Type,Percentage of Sales,week_of_year,year,Year Week Number,id,Scent,Product,Total Scent Sales,Sales
0,0c60c126,2020-01-06,0.5l,Liquid,0.33,2,2020,202002,0c60c1260.5l,Orange,0c60c1260.5l,3537.73,1167.45
1,0c60c126,2020-01-06,100g,Bar,0.13,2,2020,202002,0c60c126100g,Orange,0c60c126100g,3537.73,459.90
2,0c60c126,2020-01-06,250ml,Liquid,0.20,2,2020,202002,0c60c126250ml,Orange,0c60c126250ml,3537.73,707.55
3,0c60c126,2020-01-06,50g,Bar,0.34,2,2020,202002,0c60c12650g,Orange,0c60c12650g,3537.73,1202.83
4,0c60c126,2020-01-13,0.5l,Liquid,0.80,3,2020,202003,0c60c1260.5l,Orange,0c60c1260.5l,8264.29,6611.43
5,0c60c126,2020-01-13,100g,Bar,0.04,3,2020,202003,0c60c126100g,Orange,0c60c126100g,8264.29,330.57
6,0c60c126,2020-01-13,50g,Bar,0.16,3,2020,202003,0c60c12650g,Orange,0c60c12650g,8264.29,1322.29
7,0c60c126,2020-01-20,0.5l,Liquid,0.60,4,2020,202004,0c60c1260.5l,Orange,0c60c1260.5l,9981.68,5989.01
8,0c60c126,2020-01-20,100g,Bar,0.30,4,2020,202004,0c60c126100g,Orange,0c60c126100g,9981.68,2994.50
9,0c60c126,2020-01-20,250ml,Liquid,0.06,4,2020,202004,0c60c126250ml,Orange,0c60c126250ml,9981.68,598.90


In [55]:
final_output = merge_2[["Year Week Number", "Scent", "Size", "Product Type", "Sales"]]

In [56]:
final_output.sort_values(["Year Week Number", "Scent"]).head(10)

,Year Week Number,Scent,Size,Product Type,Sales
38,202002,Coconut,0.5l,Liquid,0.20
39,202002,Coconut,100g,Bar,0.81
40,202002,Coconut,250ml,Liquid,15.28
41,202002,Coconut,50g,Bar,4.07
115,202002,Honey,0.5l,Liquid,1141.49
116,202002,Honey,100g,Bar,449.68
117,202002,Honey,250ml,Liquid,691.81
118,202002,Honey,50g,Bar,1176.08
153,202002,Lavendar,0.5l,Liquid,18.70
154,202002,Lavendar,100g,Bar,74.79


In [58]:
# validate the row count
final_output.shape

(307, 5)